In [1]:
import pennylane as qml
from pennylane import numpy as pnp
import numpy as np
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate
from scipy.optimize import minimize
import os
import json
from susy_qm import calculate_Hamiltonian

In [2]:
potential = 'AHO'
cutoff = 4
shots = None

In [3]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)
eigenvalues = np.sort(np.linalg.eig(H)[0])
min_eigenvalue = min(eigenvalues.real)

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [4]:
min_eigenvalue

np.float64(-0.16478526068502214)

In [5]:
pauli_terms = list(zip(hamiltonian.paulis, hamiltonian.coeffs))
pauli_terms

[(Pauli('III'), np.complex128(10.312499999999996+0j)),
 (Pauli('IXI'), np.complex128(6.882221512309609+0j)),
 (Pauli('IXZ'), np.complex128(-1.8440856963554602+0j)),
 (Pauli('IZI'), np.complex128(-4.062499999999998+0j)),
 (Pauli('IZZ'), np.complex128(-4.0625+0j)),
 (Pauli('ZII'), np.complex128(2.75+0j)),
 (Pauli('ZXI'), np.complex128(1.4488887394336023+0j)),
 (Pauli('ZXZ'), np.complex128(-0.38822856765378133+0j)),
 (Pauli('ZZI'), np.complex128(-0.7500000000000001+0j)),
 (Pauli('ZZZ'), np.complex128(-0.7500000000000001+0j))]

In [102]:
def get_ansatz(evolution_time, num_trotter_steps = 10):
        
        dev = qml.device("default.qubit", wires=num_qubits)
        
        @qml.qnode(dev)
        def ansatz():

            for wire in range(num_qubits):
                 qml.PauliX(wire)

            for _ in range(num_trotter_steps):
                for term, coeff in pauli_terms:
                    pauli_word = term.to_label()
                    qml.PauliRot(100 * coeff * evolution_time / num_trotter_steps, pauli_word, wires=list(range(num_qubits)))
            return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))
        
        return ansatz

In [103]:
def cost_function(evolution_time):
        ansatz = get_ansatz(evolution_time)
        return ansatz()

In [108]:
def compute_gradients(evolution_time):
        grad_fn = qml.grad(cost_function)
        print(grad_fn(evolution_time))
        return grad_fn(evolution_time)

In [119]:
def optimize_time_evolution(evolution_time = 1.0, o_iters=200, learning_rate=0.01):

        opt = qml.AdaptiveOptimizer(stepsize=learning_rate)    
        evolution_time = pnp.tensor(evolution_time, requires_grad=True)

        for i in range(o_iters):
            circuit, energy, evolution_time = opt.step_and_cost(compute_gradients(evolution_time))
            if i % 10 == 0:
                print(f"Step {i}: Evolution time = {evolution_time}")

        optimized_time = evolution_time
        print(f"Optimized evolution time: {optimized_time}")
        return optimized_time

In [120]:
print("Initial Cost: ", cost_function(1.0))
print("Cost at 10x evolution time: ", cost_function(10.0))
print("Cost at 0.1x evolution time: ", cost_function(0.1))


Initial Cost:  1.0000000000000018
Cost at 10x evolution time:  1.0000000000000009
Cost at 0.1x evolution time:  0.9999999999999992


In [121]:
optimized_time = optimize_time_evolution()
optimized_energy = cost_function(optimized_time)
print(f"Final optimized energy: {optimized_energy}")

-2.0689161379333923e-15


TypeError: AdaptiveOptimizer.step_and_cost() missing 1 required positional argument: 'operator_pool'

In [55]:
print(qml.draw(get_ansatz(optimized_time), show_matrices=False)())

0: ──X─╭RIII─╭RIZI─╭RIZZ─╭RZII─╭RIII─╭RIZI─╭RIZZ─╭RZII─╭RIII─╭RIZI─╭RIZZ─╭RZII─┤ ╭<𝓗(M0)>
1: ──X─├RIII─├RIZI─├RIZZ─├RZII─├RIII─├RIZI─├RIZZ─├RZII─├RIII─├RIZI─├RIZZ─├RZII─┤ ├<𝓗(M0)>
2: ──X─╰RIII─╰RIZI─╰RIZZ─╰RZII─╰RIII─╰RIZI─╰RIZZ─╰RZII─╰RIII─╰RIZI─╰RIZZ─╰RZII─┤ ╰<𝓗(M0)>


In [ ]:
class TimeEvolutionAnsatz:
    def __init__(self, potential, cutoff, evolution_time, num_trotter_steps=1, **kwargs):
        self.potential = potential
        self.cutoff = cutoff
        self.evolution_time = evolution_time
        self.num_trotter_steps = num_trotter_steps
        self.optimized_time = None

        # Compute Hamiltonian
        self.H = calculate_Hamiltonian(cutoff, potential)  # Assuming function is defined
        hamiltonian = SparsePauliOp.from_operator(self.H)
        self.num_qubits = hamiltonian.num_qubits

        # Get eigenvalues for comparison
        eigenvalues, eigenvectors = np.linalg.eigh(self.H)
        self.min_eigenvalue = np.min(eigenvalues)
        self.min_eigenvector = eigenvectors[:, np.argmin(eigenvalues)]
        
        # Convert Hamiltonian to Pauli terms
        self.pauli_terms = hamiltonian.terms

    def get_ansatz(self, evolution_time=None):
        """
        Returns a QNode representing the ansatz circuit using the optimized evolution time.
        """
        if evolution_time is None:
            evolution_time = self.optimized_time if self.optimized_time is not None else self.evolution_time

        dev = qml.device("default.qubit", wires=self.num_qubits)
        
        @qml.qnode(dev)
        def ansatz():
            for _ in range(self.num_trotter_steps):
                for term, coeff in zip(*self.pauli_terms):
                    qml.PauliRot(-2 * coeff * evolution_time / self.num_trotter_steps, term, wires=range(self.num_qubits))
            return qml.expval(qml.Hermitian(self.H, wires=range(self.num_qubits)))
        
        return ansatz
    
    def cost_function(self, evolution_time):
        """
        Compute the cost function based on time evolution ansatz.
        """
        ansatz = self.get_ansatz(evolution_time)
        return ansatz()

    def optimize_time_evolution(self, o_iters=1000, o_tol=1e-6):
        """
        Optimize the ansatz evolution time to minimize the energy.
        """
        print("Optimizing Time Evolution Ansatz")
        
        res = minimize(
            self.cost_function,
            self.evolution_time,
            method="COBYLA",
            options={'maxiter': o_iters, 'tol': o_tol}
        )
        
        self.optimized_time = res.x
        print(f"Optimized evolution time: {self.optimized_time}")
        return self.optimized_time
    
    def run(self):
        """
        Run the full optimization and return the final ansatz.
        """
        self.optimize_time_evolution()
        optimized_energy = self.cost_function(self.optimized_time)
        print(f"Final optimized energy: {optimized_energy}")
        return self.get_ansatz(self.optimized_time)
